Họ tên: Vũ Minh Phát

MSSV: 21127739

# HW0: Làm quen với CUDA

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [1]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

## Câu 1

**Đề bài**: Viết hàm và thử nghiệm in ra các thông tin của card màn hình như sau:
- GPU card's name
- GPU computation capabilities
- Maximum number of block dimensions
- Maximum number of grid dimensions
- Maximum size of GPU memory
- Amount of constant and share memory
- Warp size

Sử dụng **nvcc** (CUDA Compiler) để biên dịch chương trình:

In [2]:
!nvcc -arch=sm_{major}{minor} ./HW0_P1.cu -o gpu_info_21127739

Sau khi biên dịch thành công, ta chạy thử nghiệm chương trình:

In [3]:
!./gpu_info_21127739

Number of CUDA-capable devices: 1

Device 0:
===== GPU Information =====
GPU name: Tesla T4
Compute capability: 7.5
Maximum block dimensions: (1024, 1024, 64)
Maximum grid dimensions: (2147483647, 65535, 65535)
Total global memory: 14.75 GB
Total constant memory: 64.00 KB
Total shared memory per block: 48.00 KB
Total shared memory per multiprocessor: 64.00 KB
Warp size: 32



> **Tài liệu tham khảo**:  
> [1] Thông tin của cấu trúc `cudaDeviceProp` - [docs.nvidia.com](https://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html).  
> [2] Gist "Get Information about CUDA cards on your system" by Steven Borrelli - [gist.github.com/stevendborrelli](https://gist.github.com/stevendborrelli/4286842).  

## Câu 2

**Đề bài**:
- Hãy viết chương trình cộng hai vector. Tuy nhiên, mỗi thread sẽ thực hiện hai phép tính cộng trên hai phần tử của mảng thay vì một phần tử như trong bài giảng lý thuyết (và file demo `01-AddVector.cu`).

- Cài đặt cả hai version được yêu cầu trong bài tập, thực hiện thử nghiệm với kích thước block `256`, biên dịch file code và chạy với các kích thước mảng `N` khác nhau. Ghi nhận thời gian chạy vào bảng kết quả tổng hợp.

Sử dụng **nvcc** (CUDA Compiler) để biên dịch chương trình:

In [4]:
!nvcc -arch=sm_{major}{minor} ./HW0_P2.cu -o vector_addition_21127739

Sau khi biên dịch thành công, ta chạy thử nghiệm chương trình:

In [5]:
!./vector_addition_21127739

===== GPU Information =====
GPU name: Tesla T4
GPU compute capability: 7.5

________[ Running test cases ]________
Vector size N = 1 ... passed
Vector size N = 64 ... passed
Vector size N = 256 ... passed
Vector size N = 1024 ... passed
Vector size N = 4096 ... passed
Vector size N = 16384 ... passed
Vector size N = 65536 ... passed
Vector size N = 262144 ... passed
Vector size N = 1048576 ... passed
Vector size N = 4194304 ... passed
Vector size N = 16777216 ... passed
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾

[===== Result summary =====]
*-------------*-------------------*-------------------*-------------------*
| Vector size |  Host time (ms)   | Device time (ms)  | Device time (ms)  |
|             |                   |    (Version 1)    |    (Version 2)    |
*-------------+-------------------+-------------------+-------------------*
| 64          | 0.009             | 0.022             | 0.029             |
| 256         | 0.009             | 0.020             | 0.020             |


Bảng kết quả tổng hợp sau khi chạy thử nghiệm chương trình cộng hai vector với các thông số thiết lập được mô tả trong yêu cầu:

| Vector size |  Host time (ms)   | Device time (ms)<br>(Version 1)  | Device time (ms)<br>(Version 2)  |
| --- | --: | --: | --: |
| 64          | 0.009             | 0.022             | 0.029             |
| 256         | 0.009             | 0.020             | 0.020             |
| 1024        | 0.011             | 0.020             | 0.019             |
| 4096        | 0.029             | 0.022             | 0.018             |
| 16384       | 0.090             | 0.021             | 0.020             |
| 65536       | 0.360             | 0.025             | 0.024             |
| 262144      | 1.410             | 0.033             | 0.034             |
| 1048576     | 5.695             | 0.075             | 0.085             |
| 4194304     | 22.187            | 0.221             | 0.283             |
| 16777216    | 88.442            | 0.800             | 1.019             |

**Nhận xét**:
- Nhìn chung, nếu chỉ xét đến thời gian chạy các phép tính cộng trên hai mảng (không kể đến thời gian: cấp phát, sao chép, hủy, v.v. trên vùng nhớ), thì ta thấy việc chạy chương trình trên GPU (device) có thể tiết kiệm rất nhiều thời gian so với khi chạy trên CPU (host). Điều này cho thấy hiệu suất vượt trội của GPU so với CPU trong các tác vụ có khả năng song song hóa cao như việc tính toán trên mảng (vector) hay ma trận.

- Khi xét riêng hai phiên bản của chương trình cộng hai vector trên GPU, ta thấy chúng đều cho ra hiệu suất tương đồng với nhau, tuy có một chút sự khác biệt nhưng không đáng kể.

> **Tài liệu tham khảo**:  
> [1] File demo `01-AddVector.cu` được cung cấp trong môn học.  
> [2] Repository "Vector Addition (CUDA)" by Thomas Papatheodore - [github.com/olcf-tutorials](https://github.com/olcf-tutorials/vector_addition_cuda).  